In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import ast

from sklearn.preprocessing import MultiLabelBinarizer

from warnings import simplefilter
simplefilter("ignore")

In [2]:
data = pd.read_csv('all_restaurants_4.csv')
data

,name,address,coordinates,rating,reviews,categories,avg_bill,cuisine,assortment,photo_url
0,"The бык, демократичный мясной ресторан","Невский проспект, 55","{'lat': 59.932201, 'lon': 30.350266}",4.5,1231.0,['Рестораны'],Средний чек 700 ₽,['Европейская кухня'],"['Пицца', 'Шашлык', 'Бургеры', 'Стейки', 'Том-...",['https://i0.photo.2gis.com/main/branch/38/700...
1,"Commode, бар-ресторан","Невский проспект, 47","{'lat': 59.932556, 'lon': 30.347396}",4.5,1748.0,"['Рестораны', 'Бары', 'Караоке-залы', 'Банкетн...",Средний чек 1500 ₽,"['Итальянская кухня', 'Японская кухня', 'Европ...","['Стейки', 'Салаты', 'Супы', 'Мидии']",['https://i7.photo.2gis.com/main/branch/38/700...
2,"Пища династии Минь, китайская закусочная и бар","Короленко, 14","{'lat': 59.942488, 'lon': 30.350266}",4.2,78.0,"['Рестораны', 'Бары', 'Кафе']",Средний чек 1700 ₽,['Китайская кухня'],"['Лапша wok', 'Рис wok', 'Салаты', 'Супы']",['https://i9.photo.2gis.com/main/branch/38/700...
3,"Salone pasta&bar, итальянский ресторан","набережная реки Фонтанки, 30","{'lat': 59.938734, 'lon': 30.34345}",4.2,177.0,"['Рестораны', 'Бары', 'Доставка еды']",Средний чек 3500 ₽,['Итальянская кухня'],"['Пицца', 'Стейки', 'Салаты', 'Супы', 'Начос',...",['https://i5.photo.2gis.com/main/branch/38/700...
4,"Тартарбар, ресторан","Виленский переулок, 15","{'lat': 59.940025, 'lon': 30.365968}",4.6,503.0,['Рестораны'],Средний чек 2000 ₽,['Авторская кухня'],"['Стейки', 'Салаты', 'Супы', 'Блюда на гриле']",['https://i8.photo.2gis.com/main/branch/38/700...
...,...,...,...,...,...,...,...,...,...,...
718,"Beer House, сеть баров","Маяковского, 4","{'lat': 59.932782, 'lon': 30.353957}",4.7,96.0,"['Бары', 'Доставка еды', 'Рестораны']",NaN,[],[],['https://i3.photo.2gis.com/main/branch/38/700...
719,"Beer House, сеть баров","улица Марата, 2","{'lat': 59.931151, 'lon': 30.355589}",4.5,367.0,"['Бары', 'Доставка еды', 'Рестораны', 'Кафе']",NaN,[],[],['https://i4.photo.2gis.com/main/branch/38/700...
720,"Beer House, сеть баров","Невский проспект, 124","{'lat': 59.930773, 'lon': 30.365124}",4.6,137.0,"['Бары', 'Доставка еды', 'Рестораны', 'Кафе']",NaN,[],[],['https://i7.photo.2gis.com/main/branch/38/700...
721,"Beer House, сеть баров","улица Некрасова, 25","{'lat': 59.938659, 'lon': 30.359389}",4.6,314.0,"['Бары', 'Доставка еды', 'Рестораны', 'Кафе']",NaN,[],[],['https://i4.photo.2gis.com/main/branch/38/534...


In [3]:
before = data.shape[0]
after = data.drop_duplicates().shape[0]
print(f'Удалено дубликатов : {(before - after)}')

Удалено дубликатов : 134


In [4]:
len(data['coordinates'].unique())

453

##### Оставляем фичи для ранжирования 

In [5]:
imp_features = ['name', 'rating', 'reviews', 'avg_bill', 'cuisine']

data_imp = data[imp_features].copy(deep=True)
data_imp

,name,rating,reviews,avg_bill,cuisine
0,"The бык, демократичный мясной ресторан",4.5,1231.0,Средний чек 700 ₽,['Европейская кухня']
1,"Commode, бар-ресторан",4.5,1748.0,Средний чек 1500 ₽,"['Итальянская кухня', 'Японская кухня', 'Европ..."
2,"Пища династии Минь, китайская закусочная и бар",4.2,78.0,Средний чек 1700 ₽,['Китайская кухня']
3,"Salone pasta&bar, итальянский ресторан",4.2,177.0,Средний чек 3500 ₽,['Итальянская кухня']
4,"Тартарбар, ресторан",4.6,503.0,Средний чек 2000 ₽,['Авторская кухня']
...,...,...,...,...,...
718,"Beer House, сеть баров",4.7,96.0,NaN,[]
719,"Beer House, сеть баров",4.5,367.0,NaN,[]
720,"Beer House, сеть баров",4.6,137.0,NaN,[]
721,"Beer House, сеть баров",4.6,314.0,NaN,[]


##### Удаляем пропущенные значения

In [6]:
data_imp = data_imp.replace('[]', np.nan)
data_imp

,name,rating,reviews,avg_bill,cuisine
0,"The бык, демократичный мясной ресторан",4.5,1231.0,Средний чек 700 ₽,['Европейская кухня']
1,"Commode, бар-ресторан",4.5,1748.0,Средний чек 1500 ₽,"['Итальянская кухня', 'Японская кухня', 'Европ..."
2,"Пища династии Минь, китайская закусочная и бар",4.2,78.0,Средний чек 1700 ₽,['Китайская кухня']
3,"Salone pasta&bar, итальянский ресторан",4.2,177.0,Средний чек 3500 ₽,['Итальянская кухня']
4,"Тартарбар, ресторан",4.6,503.0,Средний чек 2000 ₽,['Авторская кухня']
...,...,...,...,...,...
718,"Beer House, сеть баров",4.7,96.0,NaN,NaN
719,"Beer House, сеть баров",4.5,367.0,NaN,NaN
720,"Beer House, сеть баров",4.6,137.0,NaN,NaN
721,"Beer House, сеть баров",4.6,314.0,NaN,NaN


In [7]:
data_imp_filled = data_imp.dropna()
data_imp_filled

,name,rating,reviews,avg_bill,cuisine
0,"The бык, демократичный мясной ресторан",4.5,1231.0,Средний чек 700 ₽,['Европейская кухня']
1,"Commode, бар-ресторан",4.5,1748.0,Средний чек 1500 ₽,"['Итальянская кухня', 'Японская кухня', 'Европ..."
2,"Пища династии Минь, китайская закусочная и бар",4.2,78.0,Средний чек 1700 ₽,['Китайская кухня']
3,"Salone pasta&bar, итальянский ресторан",4.2,177.0,Средний чек 3500 ₽,['Итальянская кухня']
4,"Тартарбар, ресторан",4.6,503.0,Средний чек 2000 ₽,['Авторская кухня']
...,...,...,...,...,...
687,"Лавашная Хачапурная, кулинария",4.3,16.0,Средний чек 380 ₽,"['Узбекская кухня', 'Восточная кухня']"
712,"Блок, мясной ресторан",4.4,96.0,Средний чек 4500 ₽,"['Русская кухня', 'Европейская кухня']"
713,"Мясонская ложа, стейк-бар",4.8,578.0,Средний чек 1500 ₽,"['Европейская кухня', 'Американская кухня']"
714,"Myasoedoff, гриль-бар",4.7,216.0,Средний чек 550 ₽,"['Кавказская кухня', 'Европейская кухня', 'Аме..."


##### Выводим средний чек

In [8]:
def extract_number(text):
    # Ищем число в строке с помощью регулярного выражения
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())  # Возвращаем найденное число
    return None  # Если число не найдено, возвращаем None

In [9]:
data_imp_filled['avg_bill'] = data_imp_filled['avg_bill'].apply(extract_number)

data_imp_filled

,name,rating,reviews,avg_bill,cuisine
0,"The бык, демократичный мясной ресторан",4.5,1231.0,700,['Европейская кухня']
1,"Commode, бар-ресторан",4.5,1748.0,1500,"['Итальянская кухня', 'Японская кухня', 'Европ..."
2,"Пища династии Минь, китайская закусочная и бар",4.2,78.0,1700,['Китайская кухня']
3,"Salone pasta&bar, итальянский ресторан",4.2,177.0,3500,['Итальянская кухня']
4,"Тартарбар, ресторан",4.6,503.0,2000,['Авторская кухня']
...,...,...,...,...,...
687,"Лавашная Хачапурная, кулинария",4.3,16.0,380,"['Узбекская кухня', 'Восточная кухня']"
712,"Блок, мясной ресторан",4.4,96.0,4500,"['Русская кухня', 'Европейская кухня']"
713,"Мясонская ложа, стейк-бар",4.8,578.0,1500,"['Европейская кухня', 'Американская кухня']"
714,"Myasoedoff, гриль-бар",4.7,216.0,550,"['Кавказская кухня', 'Европейская кухня', 'Аме..."


##### Кодируем кухни

In [10]:
#pip install ast

In [11]:
# Функция для преобразования строки в список и извлечения кухонь
def extract_cuisines(cuisine_str):
    try:
        # Преобразуем строку в список
        cuisine_list = ast.literal_eval(cuisine_str)
        # Объединяем элементы списка в строку через запятую
        return ', '.join(cuisine_list)
    except (ValueError, SyntaxError):
        # Если преобразование не удалось, возвращаем исходное значение
        return cuisine_str

# Применяем функцию к столбцу
data_imp_filled['cuisine'] = data_imp_filled['cuisine'].apply(extract_cuisines)
data_imp_filled

,name,rating,reviews,avg_bill,cuisine
0,"The бык, демократичный мясной ресторан",4.5,1231.0,700,Европейская кухня
1,"Commode, бар-ресторан",4.5,1748.0,1500,"Итальянская кухня, Японская кухня, Европейская..."
2,"Пища династии Минь, китайская закусочная и бар",4.2,78.0,1700,Китайская кухня
3,"Salone pasta&bar, итальянский ресторан",4.2,177.0,3500,Итальянская кухня
4,"Тартарбар, ресторан",4.6,503.0,2000,Авторская кухня
...,...,...,...,...,...
687,"Лавашная Хачапурная, кулинария",4.3,16.0,380,"Узбекская кухня, Восточная кухня"
712,"Блок, мясной ресторан",4.4,96.0,4500,"Русская кухня, Европейская кухня"
713,"Мясонская ложа, стейк-бар",4.8,578.0,1500,"Европейская кухня, Американская кухня"
714,"Myasoedoff, гриль-бар",4.7,216.0,550,"Кавказская кухня, Европейская кухня, Американс..."


In [12]:
#from sklearn.preprocessing import MultiLabelBinarizer

In [13]:
unique_cuisines = set()
for cuisine_str in data_imp_filled['cuisine']:
    # Разделяем строку по запятой и убираем лишние пробелы
    cuisines = [c.strip() for c in cuisine_str.split(',')]
    # Добавляем кухни в множество уникальных значений
    unique_cuisines.update(cuisines)

# Преобразуем множество в отсортированный список
unique_cuisines = sorted(unique_cuisines)

# Выводим результат
print(f"Число уникальных кухонь: {len(unique_cuisines)}")
unique_cuisines

Число уникальных кухонь: 30


['Авторская кухня',
 'Азиатская кухня',
 'Американская кухня',
 'Веганская кухня',
 'Вегетарианская кухня',
 'Восточная кухня',
 'Вьетнамская кухня',
 'Греческая кухня',
 'Грузинская кухня',
 'Домашняя кухня',
 'Европейская кухня',
 'Израильская кухня',
 'Испанская кухня',
 'Итальянская кухня',
 'Кавказская кухня',
 'Китайская кухня',
 'Корейская кухня',
 'Мексиканская кухня',
 'Немецкая кухня',
 'Паназиатская кухня',
 'Русская кухня',
 'Сербская кухня',
 'Средиземноморская кухня',
 'Тайская кухня',
 'Турецкая кухня',
 'Узбекская кухня',
 'Украинская кухня',
 'Французская кухня',
 'Халяльная кухня',
 'Японская кухня']

In [14]:
# Создаем столбцы для каждой уникальной кухни и заполняем их нулями
for cuisine in unique_cuisines:
    data_imp_filled[cuisine] = 0

for index, row in data_imp_filled.iterrows():
    # Преобразуем строку в список кухонь, разделяя по запятой
    cuisines = [c.strip() for c in row['cuisine'].split(',')]
    for cuisine in cuisines:
        # Убираем лишние пробелы и проверяем, что кухня есть в unique_cuisines
        cuisine_cleaned = cuisine.strip()
        if cuisine_cleaned in unique_cuisines:
            data_imp_filled.at[index, cuisine_cleaned] = 1

In [15]:
data_imp_filled

,name,rating,reviews,avg_bill,cuisine,Авторская кухня,Азиатская кухня,Американская кухня,Веганская кухня,Вегетарианская кухня,...,Русская кухня,Сербская кухня,Средиземноморская кухня,Тайская кухня,Турецкая кухня,Узбекская кухня,Украинская кухня,Французская кухня,Халяльная кухня,Японская кухня
0,"The бык, демократичный мясной ресторан",4.5,1231.0,700,Европейская кухня,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Commode, бар-ресторан",4.5,1748.0,1500,"Итальянская кухня, Японская кухня, Европейская...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Пища династии Минь, китайская закусочная и бар",4.2,78.0,1700,Китайская кухня,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Salone pasta&bar, итальянский ресторан",4.2,177.0,3500,Итальянская кухня,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Тартарбар, ресторан",4.6,503.0,2000,Авторская кухня,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,"Лавашная Хачапурная, кулинария",4.3,16.0,380,"Узбекская кухня, Восточная кухня",0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
712,"Блок, мясной ресторан",4.4,96.0,4500,"Русская кухня, Европейская кухня",0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
713,"Мясонская ложа, стейк-бар",4.8,578.0,1500,"Европейская кухня, Американская кухня",0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
714,"Myasoedoff, гриль-бар",4.7,216.0,550,"Кавказская кухня, Европейская кухня, Американс...",0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
